In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from segmentation_models import get_preprocessing
from segmentation_models.losses import DiceLoss, CategoricalCELoss
from segmentation_models.metrics import iou_score, f1_score
from segmentation_models import Unet
from tensorflow.keras.optimizers.experimental import AdamW
from keras.metrics import BinaryIoU
import numpy as np
from tensorflow import keras
from PIL import Image

In [ ]:
# Definir parâmetros

train_images_dir = "Datasets/Combined_CPA_BuildingsOnly_BlurPequeno7/train/images"
train_masks_dir = "Datasets/Combined_CPA_BuildingsOnly_BlurPequeno7/train/labels"

val_images_dir = "Datasets/Combined_CPA_BuildingsOnly_BlurPequeno7/val/images"
val_masks_dir = "Datasets/Combined_CPA_BuildingsOnly_BlurPequeno7/val/labels"

test_images_dir = "Datasets/PrivateDataset_BuildingsOnly/test/images"
test_masks_dir = "Datasets/PrivateDataset_BuildingsOnly/test/labels"

BACKBONE = 'efficientnetb6'
preprocess_input = get_preprocessing(BACKBONE)
img_size = (512, 512)
batch_size = 6
num_classes = 1

imagenet_mean = tf.constant([0.485, 0.456, 0.406], shape=[1, 1, 3], dtype=tf.float32)
imagenet_std  = tf.constant([0.229, 0.224, 0.225], shape=[1, 1, 3], dtype=tf.float32)

In [ ]:
# Carregar imagens de entrada (sem rótulo, pois as máscaras são carregadas separadamente)
train_images = tf.keras.utils.image_dataset_from_directory(
    train_images_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode=None,
    seed=1335
)

# Carregar máscaras (as máscaras estão em escala de cinza)
train_masks = tf.keras.utils.image_dataset_from_directory(
    train_masks_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale",
    label_mode=None,
    seed=1335
)

# Carregar conjunto de validação
val_images = tf.keras.utils.image_dataset_from_directory(
    val_images_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode=None,
    seed=1336
)

val_masks = tf.keras.utils.image_dataset_from_directory(
    val_masks_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale",
    label_mode=None,
    seed=1336
)

# Carregar conjunto de teste
test_images = tf.keras.utils.image_dataset_from_directory(
    test_images_dir,
    image_size=img_size,
    batch_size=batch_size,
    label_mode=None,
    seed=1337
)

test_masks = tf.keras.utils.image_dataset_from_directory(
    test_masks_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode="grayscale",
    label_mode=None,
    seed=1337
)

# Normalizar imagens e converter máscaras para categorias
def preprocess(images, masks):
    # Normalizar imagens
    images = tf.cast(images, tf.float32) / 255.0
    images = (images - imagenet_mean) / imagenet_std

    # Converter máscaras para binário e normalizar
    masks = tf.cast(masks, tf.float32)
    masks = tf.squeeze(masks, axis=-1)  # Remover canais extras
    masks = tf.expand_dims(masks, axis=-1)  # Garantir que o formato seja (altura, largura, 1)

    return images, masks

# Criar datasets zipando imagens e máscaras
train_ds = tf.data.Dataset.zip((train_images, train_masks)).map(preprocess)
val_ds = tf.data.Dataset.zip((val_images, val_masks)).map(preprocess)
test_ds = tf.data.Dataset.zip((test_images, test_masks)).map(preprocess)

In [ ]:
for images, masks in test_ds.take(1):  # Pegando apenas 1 batch para exemplo
    # Imprimir as máscaras
    print("Máscaras do batch:")
    print(masks.numpy())  # Converte o tensor para um numpy array para exibição

    # Se você quiser visualizar o formato das máscaras:
    print("Formato das máscaras:", masks.shape)

    # Se quiser visualizar as primeiras máscaras de um batch específico:
    print("Primeira máscara do batch:")
    print(masks[0].numpy())  # Imprime a primeira máscara do batch

In [ ]:
model = Unet(BACKBONE, input_shape=(512, 512, 3), encoder_weights='imagenet')

tf.keras.backend.clear_session()

# 🔹 Compilar o modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[BinaryIoU(), f1_score]  # IoU e F1 Score diretamente do segmentation_models
)

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "uNetB6BuildingsOnlyCombinedCPA_BlurPequeno7.keras",
        save_best_only=True,  # Salva apenas o melhor modelo
        monitor="val_loss",  # Monitorar a perda de validação
        mode="min"
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,  # Para em 20 épocas sem melhora
        restore_best_weights=True
    )
]
# Treinar modelo
history = model.fit(
    train_ds,
    epochs=50,
    callbacks=callbacks,
    validation_data=val_ds
)

In [ ]:
history_dict = history.history

# Plotando a loss
plt.figure(figsize=(12, 5))
plt.plot(history_dict['loss'], label='Training Loss')
plt.plot(history_dict['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plotando a acurácia
plt.figure(figsize=(12, 5))
plt.plot(history_dict['accuracy'], label='Training Accuracy')
plt.plot(history_dict['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plotando a acurácia
plt.figure(figsize=(12, 5))
plt.plot(history_dict['iou_score'], label='Training IoU')
plt.plot(history_dict['val_iou_score'], label='Validation IoU')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
custom_objects = {
    'f1-score': f1_score
}

model = tf.keras.models.load_model("uNetB6BuildingsOnlyCombinedCPA_BlurPequeno7.keras", custom_objects=custom_objects)

#model.summary()

In [ ]:
def predict_and_plot_mask(image_path, model, img_size, imagenet_mean, imagenet_std, threshold=0.5, save_mask_path="predicted_mask.png"):
    """
    Carrega a imagem, realiza a normalização, faz a predição da máscara, exibe os resultados e salva a máscara predita.

    Parâmetros:
      - image_path: Caminho para a imagem de teste.
      - model: Modelo treinado (por exemplo, uma rede neural Keras).
      - img_size: Tupla com o tamanho desejado (altura, largura) para a imagem.
      - imagenet_mean: Média para normalização (ex.: [0.485, 0.456, 0.406]).
      - imagenet_std: Desvio padrão para normalização (ex.: [0.229, 0.224, 0.225]).
      - threshold: Limiar para segmentação binária da máscara (default=0.5).
      - save_mask_path: Caminho para salvar a máscara predita.
    """
    # Carrega a imagem de teste com o tamanho especificado
    img = keras.utils.load_img(image_path, target_size=img_size)

    # Converte a imagem para array e normaliza para [0, 1]
    img_array1 = keras.utils.img_to_array(img)
    img_array = img_array1 / 255.0

    # Normalização usando os parâmetros do ImageNet (se necessário)
    img_array = (img_array - imagenet_mean) / imagenet_std

    # Adiciona uma dimensão para formar um batch (1, altura, largura, canais)
    img_array = np.expand_dims(img_array, axis=0)

    # Faz a predição com o modelo
    predictions = model.predict(img_array)

    # Para uma saída binária, extraímos a primeira camada da previsão (assumindo 1 canal)
    predictions = predictions[0, :, :, 0]

    # Limiarização para obter a máscara binária (0 ou 1)
    predictions = (predictions > threshold).astype(np.uint8)

    # Exibe a imagem original, a máscara predita e a sobreposição
    plt.figure(figsize=(18, 6))

    # Imagem original
    plt.subplot(1, 3, 1)
    plt.title('Test Image')
    plt.imshow(img_array1.astype(np.uint8))
    plt.axis("off")

    # Máscara predita
    plt.subplot(1, 3, 2)
    plt.title('Predicted Mask')
    plt.imshow(predictions, cmap='gray')
    plt.axis("off")

    # Overlay da imagem com a máscara (máscara em vermelho)
    plt.subplot(1, 3, 3)
    plt.title('Overlay of Image and Mask')
    red_mask = np.zeros_like(img_array1)
    red_mask[:, :, 0] = predictions * 255  # Canal vermelho recebe a máscara
    overlay = img_array1 * 0.7 + red_mask * 0.3
    plt.imshow(overlay.astype(np.uint8))
    plt.axis("off")

    plt.show()

    # Converte a máscara para escala [0, 255] e salva como imagem
    mask = (predictions * 255).astype(np.uint8)
    mask_img = Image.fromarray(mask)
    mask_img.save(save_mask_path)

In [ ]:
predict_and_plot_mask("AerialImageDataset/test/images/bellingham1_slice_1.png", model, img_size=(512,512),
                       imagenet_mean=[0.485, 0.456, 0.406], imagenet_std=[0.229, 0.224, 0.225])

In [ ]:
# Avaliação do modelo no conjunto de teste
(TotalLoss, IoU, f1_score) = model.evaluate(test_ds)  # Pegando todos os valores retornados
print("TotalLoss: ", TotalLoss)  # Isso imprimirá todos os valores retornados
print("IoU: ", IoU)  # Isso imprimirá todos os valores retornados
print("F1: ", f1_score)  # Isso imprimirá todos os valores retornados